In [2]:
import numpy as np
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from Mesh.Charges_utils import get_charges_list
from Model.Solutions_utils import Solution_utils
from Simulation import get_simulation_name

simulation_name,results_path,main_path,logger = get_simulation_name(os.path.join(os.path.dirname(os.getcwd()), 'dataset.py'))

file_path = os.getcwd()

In [3]:

molecule = 'arg'


Rs = {'born_ion': 1,
      'methanol': 2.54233,
      'methanol_E': 2.54233,
      'arg': 5.9695745,
      'sphere': 1.2,
      'sphere_+1-1': 1.2,
      'sphere_+1+1': 1.2,
      'sphere_+2-1': 1.2,
      'sphere_+1_D': 1.2,
      'sphere_0.2': 1.2}

Model = Solution_utils()

Model.pqr_path = os.path.join(os.getcwd(),molecule,f'{molecule}.pqr')
q_list = get_charges_list(Model.pqr_path)

Model.domain_properties = {
        'molecule': molecule,
        'epsilon_1':  2,
        'epsilon_2': 80,
        'kappa': 0.125,
        'T' : 300 
        }

Model.epsilon_1 = Model.domain_properties['epsilon_1']
Model.epsilon_2 = Model.domain_properties['epsilon_2']
Model.kappa = Model.domain_properties['kappa']
Model.pi = np.pi
Model.q_list = q_list

n = len(Model.q_list)
Model.qs = np.zeros(n)
Model.x_qs = np.zeros((n,3))
for i,q in enumerate(Model.q_list):
    Model.qs[i] = q.q
    Model.x_qs[i,:] = q.x_q
Model.total_charge = np.sum(Model.qs)

dR = 0.1
R_mol = Rs[molecule]
Rmin = dR + R_mol
Rmax = Rmin + 3

Model.mesh_density = 3
Model.mesh_generator = 'msms'
Model.DTYPE = 'float64'

In [4]:
q_list

[Charges(q=0.7214, x_q=array([ 0.392,  0.877, -2.219]), r_q=1.908, atom_name='C', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=-0.6013, x_q=array([ 1.262,  0.206, -2.769]), r_q=1.6612, atom_name='O', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.2083, x_q=array([ 0.586,  1.668, -3.938]), r_q=0.6, atom_name='H', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.2083, x_q=array([ 0.459,  3.157, -3.268]), r_q=0.6, atom_name='H2', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.2083, x_q=array([ 1.672,  2.154, -2.813]), r_q=0.6, atom_name='H3', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.1305, x_q=array([-0.126, -1.073, -0.834]), r_q=1.824, atom_name='N', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=0.2083, x_q=array([ 0.149, -1.142,  0.125]), r_q=0.6, atom_name='H', res_name='ARG', res_num='2', ion_r_explode=3.5),
 Charges(q=-0.0223, x_q=array([-0.486,  0.311, -1.129]), r_q=1.908, atom_name='CA', res

### Calculate Solvation energy

In [5]:
phi_q = Model.Harmonic_spheres(Model.x_qs,'molecule',R_mol)
print(Model.solvation_energy_phi_qs(phi_q))
print(phi_q)
Model.solvation_energy_phi_qs(phi_q)

-73.13704757700724
[-0.0149601  -0.01451289 -0.0165293  -0.01756816 -0.01609127 -0.01400082
 -0.01420401 -0.01449526 -0.01436105 -0.01478346 -0.01487474 -0.01501364
 -0.01465445 -0.01440891 -0.01448955 -0.01504634 -0.01500428 -0.01502074
 -0.01554073 -0.01578378 -0.01580813 -0.01551957 -0.01506542 -0.01584317
 -0.01652246 -0.01676533 -0.01704408]


-73.13704757700724

### Create data_known file

In [6]:

def calculate_print_phi(domain,method, num_points, file):

    R = R_mol if domain == 'molecule' else Rmax if domain == 'solvent' else None
    x = np.random.uniform(-R, R, num_points)
    y = np.random.uniform(-R, R, num_points)
    z = np.random.uniform(-R, R, num_points)
    r = np.sqrt(x**2+y**2+z**2)
    X = np.stack([x,y,z], axis=1)

    phi_values = Model.phi_known(method,'phi',X,domain,R_mol)

    if domain == 'molecule':
        for i in range(num_points):
            condition = 1 if r[i] <= R_mol else 2
            if r[i]<R_mol:
                file.write(f"{condition} {x[i]} {y[i]} {z[i]} {phi_values[i]}\n")

    elif domain == 'solvent':
        for i in range(num_points):
            condition = 1 if r[i] <= R_mol else 2
            if r[i]>Rmin:
                file.write(f"{condition} {x[i]} {y[i]} {z[i]} {phi_values[i]}\n")

In [7]:
with open(os.path.join(file_path,molecule,'data_known.dat'), 'w') as file:
    calculate_print_phi('solvent','pbj',300, file)
    # calculate_print_phi('molecule','Harmonic_spheres',300, file)

print('File created')

/home/martinachondo/PINN/code/Model/pbj/mesh/ExternalSoftware/MSMS/msms -if /home/martinachondo/PINN/code/Molecules/mesh_temp/arg.xyzr -of /home/martinachondo/PINN/code/Molecules/mesh_temp/arg -p 1.4 -d 5 -no_header
MSMS 2.6.1 started on tulio
Copyright M.F. Sanner (1994)
Compilation flags -O2 -DVERBOSE -DTIMING
INPUT  /home/martinachondo/PINN/code/Molecules/mesh_temp/arg.xyzr 27 spheres 0 collision only, radii  0.600 to  1.908
PARAM  Probe_radius  1.400 density  5.000 hdensity  3.000
REDUCED SURFACE ...
    RS component 0 identified 13 14 12
    0 free edge(s) found in component 0
    RS component  #faces  #edges #free_edges  #vertices   genus
         0          42       63          0         20      1.5
    Time Reduced Surface real: 0.00 user: 0.00 sys: 0.00
ANALYTICAL SOLVENT EXCLUDED SURFACE...
    Component   0
        Time Surface real: 0.00 user: 0.00 sys: 0.00
        Time Singularities real: 0.00 user: 0.00 sys: 0.00
    SES comp.  #fac.  #edg. #s_edg. #vert. #s_vert. R_h C_

File created
